In [ ]:
!pip install category_encoders
!pip install scipy
!pip install statsmodels


In [ ]:
import pandas as pd

# Импортируем объект Counter из модуля collections
from collections import Counter
import re
import numpy as np
from sklearn import preprocessing
import category_encoders as ce

import matplotlib.pyplot as plt
import seaborn as sns
import os.path


# Для стат тестов импортируем библиотеки и сразу присвоим уровень значимости
import scipy.stats as stats
alpha=0.05


# Загрузка Данных

In [ ]:
if os.path.exists('/kaggle/input/sf-booking/hotels_train.csv')==False:
    from google.colab import drive
    drive.mount('/content/drive')

Т.к. Kaggle плохо подсказывает блокнот делался на GoogleCollab

In [ ]:
if os.path.exists('/content/hotels.csv'):
    hotels = pd.read_csv('/content/hotels.csv')

elif os.path.exists('//content/drive/MyDrive/Colab Notebooks/SkilFactory/Блок 3. Разведывательный анализ данных/Проект/hotels.csv'):
    hotels = pd.read_csv('//content/drive/MyDrive/Colab Notebooks/SkilFactory/Блок 3. Разведывательный анализ данных/Проект/hotels.csv')

elif os.path.exists('/kaggle/input/sf-booking/hotels_train.csv'):
    hotels = pd.read_csv('/kaggle/input/sf-booking/hotels_train.csv')
    
    
    
hotels.head(3)

## Проведём беглый осмотр таблиц

In [ ]:
hotels.info()

In [ ]:
# Создадим словарь список с названиями колонок
listStartCollNames=hotels.columns
listStartCollNames

In [ ]:
hotels.describe(include=[object])

In [ ]:
hotels.describe()

In [ ]:
# Найдём столбцы с пропущенными значениями

hotels.isnull().sum()[hotels.isnull().sum()>0]

# 2. Разведывательный анализ данных

# Работа с числовыми признаками

In [ ]:
# Выделим числовые признаки в отдельный список
List_intCollumns=hotels.select_dtypes(['int64', 'float64']).columns
List_intCollumns

In [ ]:
# Нарисуем для каждого Числового признака коробку Тьюкки и Гистограмму и Граффик зависимости целевого признака от текущего

fig,axes=plt.subplots(nrows=3,ncols=3,figsize=(30,10))

sns.boxplot(x=hotels['additional_number_of_scoring'],ax=axes[(0,0)])
# axes[(0,0)].boxplot(x=hotels['additional_number_of_scoring'],vert=False)
sns.histplot(x=hotels['additional_number_of_scoring'],ax=axes[(0,1)],bins=20)
sns.scatterplot(x=hotels['additional_number_of_scoring'],y=hotels['reviewer_score'],ax=axes[(0,2)])

sns.boxplot(x=hotels['average_score'],ax=axes[(1,0)])
# axes[(1,0)].boxplot(x=hotels['average_score'],vert=False)
sns.histplot(x=hotels['average_score'],ax=axes[(1,1)],bins=20)
sns.scatterplot(x=hotels['average_score'],y=hotels['reviewer_score'],ax=axes[(1,2)])

sns.boxplot(x=hotels['review_total_negative_word_counts'],ax=axes[(2,0)])
# axes[(2,0)].boxplot(x=hotels['review_total_negative_word_counts'],vert=False)
sns.histplot(x=hotels['review_total_negative_word_counts'],ax=axes[(2,1)],bins=20)
sns.scatterplot(x=hotels['review_total_negative_word_counts'],y=hotels['reviewer_score'],ax=axes[(2,2)])




In [ ]:
fig,axes=plt.subplots(nrows=3,ncols=3,figsize=(30,10))

sns.boxplot(x=hotels['total_number_of_reviews'],ax=axes[(0,0)])
#axes[(0,0)].boxplot(x=hotels['total_number_of_reviews'],vert=False)
sns.histplot(x=hotels['total_number_of_reviews'],ax=axes[(0,1)],bins=20)
sns.scatterplot(x=hotels['total_number_of_reviews'],y=hotels['reviewer_score'],ax=axes[(0,2)])

sns.boxplot(x=hotels['review_total_positive_word_counts'],ax=axes[(1,0)])
#axes[(1,0)].boxplot(x=hotels['review_total_positive_word_counts'],vert=False)
sns.histplot(x=hotels['review_total_positive_word_counts'],ax=axes[(1,1)],bins=20)
sns.scatterplot(x=hotels['review_total_positive_word_counts'],y=hotels['reviewer_score'],ax=axes[(1,2)])

sns.boxplot(x=hotels['additional_number_of_scoring'],ax=axes[(2,0)])
#axes[(2,0)].boxplot(x=hotels['additional_number_of_scoring'],vert=False)
sns.histplot(x=hotels['additional_number_of_scoring'],ax=axes[(2,1)],bins=20)
sns.scatterplot(x=hotels['additional_number_of_scoring'],y=hotels['reviewer_score'],ax=axes[(2,2)])


In [ ]:
fig,axes=plt.subplots(nrows=3,ncols=3,figsize=(30,10))

sns.boxplot(x=hotels['reviewer_score'],ax=axes[(0,0)])
#axes[(0,0)].boxplot(x=hotels['reviewer_score'],vert=False)
sns.histplot(x=hotels['reviewer_score'],ax=axes[(0,1)],bins=20)
sns.scatterplot(x=hotels['reviewer_score'],y=hotels['reviewer_score'],ax=axes[(0,2)])

sns.boxplot(x=hotels['lat'],ax=axes[(1,0)])
#axes[(1,0)].boxplot(x=hotels['lat'],vert=False)
sns.histplot(x=hotels['lat'],ax=axes[(1,1)],bins=10)
sns.scatterplot(x=hotels['lat'],y=hotels['reviewer_score'],ax=axes[(1,2)])

sns.boxplot(x=hotels['lng'],ax=axes[(2,0)])
#axes[(2,0)].boxplot(x=hotels['lng'],vert=False)
sns.histplot(x=hotels['lng'],ax=axes[(2,1)],bins=10)
sns.scatterplot(x=hotels['lng'],y=hotels['reviewer_score'],ax=axes[(2,2)])

На  граффиках scatterplot  ничего не понятно . Кроме одного  review_total_positive_word_counts   там видно что  есть  некая линейная связь

In [ ]:
# Удалим из спаиска  Числовых признаков целевой и  будем избовлятся от выбросов и

List_intCollumns=list(List_intCollumns)
List_intCollumns.remove("reviewer_score")

# Также удалим координаты отелей - там есть пропуски их можно заполнить
List_intCollumns.remove('lat')
List_intCollumns.remove('lng')



In [ ]:
# Проверим признаки на нормальность
# К нормальным применим известные методы очистки выбросов
List_NormalColl=[]
List_NotNormalColl=[]

for strCollName in List_intCollumns:


    seriesTestColl=hotels[strCollName]

    # Проведём тест Шапиро_Уика

    _,p=stats.shapiro(seriesTestColl)


    # Если признак распределён не нормально  то возьмём натуральный логарифм и попробуем снова тест Шапиро_Уика
    if p<=alpha:
      seriesTestColl=np.log(seriesTestColl+1)
      _,p=stats.shapiro(seriesTestColl)
    else:
      print(f"{strCollName} прологорифмируем и снова попробуем")

    if p<=alpha:
      print(f"{strCollName} - распределён не нормальное")
      List_NotNormalColl.append(strCollName)
    else:
      print(f"{strCollName} - распределён нормально")
      List_NormalColl.append(strCollName)


List_NormalColl

Все признаки распределены не нормально - Даже не похоже


In [ ]:
# напишем функцию котрая будет приравнивать выброс к 3ей или 1ой квантили И будет собирать массив индексов строк с выбросами

# Множество строк которые нужно удалить
set_Row_ForDell=set()

def quant_1_or_3(hotels:pd.DataFrame, CollName:str):

      SeriesColl=hotels[CollName].copy()
      quartile_1= SeriesColl.quantile(0.25)
      quartile_3 =SeriesColl.quantile(0.75)
      iqr = quartile_3 - quartile_1

      index_Min_1=(SeriesColl[SeriesColl<(quartile_1  - (1.5*iqr))   ]).index
      SeriesColl.loc[index_Min_1]=SeriesColl.quantile(0.25)
      print(f"Для {CollName} первая квантиль равна {SeriesColl.quantile(0.25)}")



      index_Max_3=(SeriesColl[SeriesColl>(quartile_3  + (1.5*iqr))   ]).index
      SeriesColl.loc[index_Max_3]=SeriesColl.quantile(0.75)
      print(f"Для {CollName} третья квантиль равна {SeriesColl.quantile(0.75)}")

      print(f"Для признака {CollName} имеется {len(index_Min_1)+ len(index_Max_3)} выбросов")

      set_Row_ForDell=set(index_Min_1)|set(index_Max_3)

      return SeriesColl,set_Row_ForDell


for CollName in List_intCollumns:
    hotels[CollName+"_Test"]=quant_1_or_3(hotels,CollName)[0]
    set_Row_ForDell=set_Row_ForDell|quant_1_or_3(hotels,CollName)[1]



## Проверим как теперь выгледят признаки с самым большим количеством выбросов
<!-- Если они корректные  значит и у остольных всё хорошо -->

In [ ]:
fig,axes=plt.subplots(nrows=2,ncols=3,figsize=(30,10))

# sns.boxplot(x=hotels['total_number_of_reviews'],ax=axes[(0,0)])
axes[(0,0)].boxplot(x=hotels['review_total_negative_word_counts_Test'],vert=False)
sns.histplot(x=hotels['review_total_negative_word_counts_Test'],ax=axes[(0,1)],bins=20)
sns.scatterplot(x=hotels['review_total_negative_word_counts_Test'],y=hotels['reviewer_score'],ax=axes[(0,2)])

# sns.boxplot(x=hotels['review_total_positive_word_counts'],ax=axes[(1,0)])
axes[(1,0)].boxplot(x=hotels['review_total_positive_word_counts_Test'],vert=False)
sns.histplot(x=hotels['review_total_positive_word_counts_Test'],ax=axes[(1,1)],bins=20)
sns.scatterplot(x=hotels['review_total_positive_word_counts_Test'],y=hotels['reviewer_score'],ax=axes[(1,2)])

Идея была неудачная. Нужно удалять выбросы. Результат модели сильно упал .

In [ ]:
# Удалим строки с выбросами

hotels.drop(list(set_Row_ForDell),axis=0,inplace=True)

In [ ]:
fig,axes=plt.subplots(nrows=2,ncols=3,figsize=(30,10))

# sns.boxplot(x=hotels['total_number_of_reviews'],ax=axes[(0,0)])
axes[(0,0)].boxplot(x=hotels['review_total_negative_word_counts'],vert=False)
sns.histplot(x=hotels['review_total_negative_word_counts'],ax=axes[(0,1)],bins=20)
sns.scatterplot(x=hotels['review_total_negative_word_counts'],y=hotels['reviewer_score'],ax=axes[(0,2)])

# sns.boxplot(x=hotels['review_total_positive_word_counts'],ax=axes[(1,0)])
axes[(1,0)].boxplot(x=hotels['review_total_positive_word_counts'],vert=False)
sns.histplot(x=hotels['review_total_positive_word_counts'],ax=axes[(1,1)],bins=20)
sns.scatterplot(x=hotels['review_total_positive_word_counts'],y=hotels['reviewer_score'],ax=axes[(1,2)])

In [ ]:
hotels.info()

Потеряли 111912  осталось 274891

In [ ]:
# Удалим тестовые Столбцы
for CollName in List_intCollumns:
    hotels.drop(CollName+"_Test",axis=1,inplace=True)

In [ ]:
fig,axes=plt.subplots(nrows=2,ncols=3,figsize=(30,10))

# sns.boxplot(x=hotels['total_number_of_reviews'],ax=axes[(0,0)])
axes[(0,0)].boxplot(x=hotels['review_total_negative_word_counts'],vert=False)
sns.histplot(x=hotels['review_total_negative_word_counts'],ax=axes[(0,1)],bins=20)
sns.scatterplot(x=hotels['review_total_negative_word_counts'],y=hotels['reviewer_score'],ax=axes[(0,2)])

# sns.boxplot(x=hotels['review_total_positive_word_counts'],ax=axes[(1,0)])
axes[(1,0)].boxplot(x=hotels['review_total_positive_word_counts'],vert=False)
sns.histplot(x=hotels['review_total_positive_word_counts'],ax=axes[(1,1)],bins=20)
sns.scatterplot(x=hotels['review_total_positive_word_counts'],y=hotels['reviewer_score'],ax=axes[(1,2)])

In [ ]:
hotels.info()

# Соединим тестовый ДатаФрейм и Тренировачный

In [ ]:
if os.path.exists('/kaggle/input/sf-booking/hotels_test.csv'):
    hotels_test = pd.read_csv('/kaggle/input/sf-booking/hotels_test.csv')
    hotels_test["TestFlag"]=1
    hotels["TestFlag"]=0
    
    hotels=pd.concat([hotels,hotels_test], ignore_index=True)


In [ ]:
hotels.info()

# Поработаем с каждым признаком отдельно

## Работа с датами

In [ ]:
# Проверим пропуски
hotels['review_date'].isnull().sum()

In [ ]:
# Преобразуем в datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date'])


In [ ]:
hotels["review_date"].min()

hotels["review_date"].max()

print( f"самый свежий отзыв {hotels['review_date'].max()}   самый первый отзыв {hotels['review_date'].min()}" )

# Поработаем с признаком tags — теги, которые рецензент дал отелю

In [ ]:

# теги находятся внутри строки которая представляет собой список - нужно для кажного тега в таблице создать столбец с бинарным признаком
# 1 найдём все уникальные теги собрав их в коллекцию

# Создадим лист для коолекции
tags_set=set()

# Напишем функцию  которая превратит строку в список
# И добавит все теги во множество






def ConvertToList(test_string :str ,tags_set:set):
  # убираем лишние конструкции
  List_add=test_string.replace("'","").replace("[","").replace("]","").strip().split(" ,  ")
  # добавляем во  множество
  # global list_add_Tags
  # list_add_Tags=list_add_Tags+List_add
  tags_set.update(set(List_add))
  return  List_add




In [ ]:
# Получаем котонку содержащую списки тегов



hotels["tags_list"]=hotels["tags"].apply(ConvertToList,tags_set=tags_set)

In [ ]:
# Добавим колонку количества тегов
hotels["Tags_Count"]=hotels["tags_list"].apply(lambda x: len(x))

In [ ]:
# Разобьём полученные списки тегов по одному
TagsHotelsDf=hotels["tags_list"].explode()
TagsHotelsDf =pd.DataFrame(TagsHotelsDf)
TagsHotelsDf["index_from_hotels"]=TagsHotelsDf.index


In [ ]:
print(f"уникальных тегов представлено в наборе данных {len(tags_set)}")
# Если для каждого тега создать свою колонку Дана сет увеличится на на 2368 колонок

In [ ]:
# Создадим таблицу справочник для обьединения с основной таблицей
modeTag=TagsHotelsDf["tags_list"].value_counts()[TagsHotelsDf["tags_list"].value_counts()==TagsHotelsDf["tags_list"].value_counts().sort_values(ascending=False).iloc[0]].index[0]
print(f"Наиболее часто встречающийся тег {modeTag}")


## Достанем признак количества ночей и узнаем на сколько ночей чаще всего останавливаются

In [ ]:
# Достанем признак количества ночей и узнаем на сколько ночей чаще всего останавливаются
NightCount=TagsHotelsDf["tags_list"].apply(lambda x : True if  re.fullmatch(r"[S,s]tayed \d{1,} [N,n]ights{0,1}", x) is not None else False)

In [ ]:
modse_Count_Night=TagsHotelsDf[NightCount]["tags_list"].value_counts().sort_values(ascending=False).index[0]

In [ ]:
modse_Count_Night_int=re.search(r"\d{1,}",modse_Count_Night)[0]
modse_Count_Night_int

In [ ]:

print(f"Чаще всего останавливаются путешественники в отелях на {modse_Count_Night_int} ночь(ночей)")


## оставим только те теги котрые встречаются в 0,5% строк

In [ ]:
TagsHotelsDf_value_count=TagsHotelsDf["tags_list"].value_counts()

In [ ]:
# Добавим в таблицу отелей признак номера строки

hotels["index_from_hotels"]=hotels.index

In [ ]:
TagsHotelsDf_value_count[TagsHotelsDf_value_count>hotels.shape[0]*0.05]

In [ ]:
# Соберём список теги которые встречаются в более чем в 5% строк

List_FamosTags=TagsHotelsDf_value_count[TagsHotelsDf_value_count>hotels.shape[0]*0.05].index
List_FamosTags


In [ ]:
# теперь добавим эти теги в таблицу отелей  методом Merge

collName='Leisure trip'

for collName in List_FamosTags:

    TegTableForMerge=TagsHotelsDf[TagsHotelsDf["tags_list"]==collName].copy()

    TegTableForMerge.loc[:,"Tags_"+collName]=1

    TegTableForMerge.drop(["tags_list"],axis=1,inplace=True)


    hotels=hotels.merge(TegTableForMerge,how='left',left_on="index_from_hotels",right_on="index_from_hotels")

    hotels.fillna({"Tags_"+collName:0},inplace=True)

# Разберём признак даты более конкретно

In [ ]:
# Достанем значение года из даты и добавим как отдельный признак

hotels["review_date_year"]=hotels["review_date"].dt.year

# получим из даты значение месяца и дня недели

hotels["review_date_day_of_week"]=hotels["review_date"].dt.strftime("%A")
hotels["review_date_month"]=hotels["review_date"].dt.strftime("%B")

In [ ]:
# Графическим путём посмотрим как распределены рецензии
df_for_graph=hotels["review_date_year"].value_counts()

fig=plt.figure(figsize=(8,4))
axes=fig.add_axes([0,0,1,1])

axes.bar(x=df_for_graph.index,height=df_for_graph)

Борльше всего рецензий было сделано в  2016 году

In [ ]:
# закодируем признаки  дня недели и названия месяца One-Hot кодированием

# День недели
encoder=ce.OneHotEncoder(cols=["review_date_day_of_week"],use_cat_names=True)

type_bin=encoder.fit_transform(hotels['review_date_day_of_week'])

hotels=pd.concat([hotels,type_bin],axis=1)


# Месяц

encoder=ce.OneHotEncoder(cols=["review_date_month"],use_cat_names=True)

type_bin=encoder.fit_transform(hotels['review_date_month'])

hotels=pd.concat([hotels,type_bin],axis=1)



# Разберём признак days_since_review

In [ ]:
# Работа с признаком days_since_review — количество дней между датой проверки и датой очистки;
hotels["days_since_review_count_day"]=hotels["days_since_review"].apply(lambda x: int( re.search(r"\d{1,}", x)[0]))


hotels["days_since_review_count_day"].describe()
# Медиана и Среденее аРЕФМЕТИЧЕСКОЕ МЕНЬШЕ МАКСИМУМА В 2 РАза
# Инициализируем нормализатор MinMaxScaler
# Мега выбросов нет

In [ ]:
mm_scaler=preprocessing.MinMaxScaler()

df_mm=mm_scaler.fit_transform(pd.DataFrame(hotels["days_since_review_count_day"]))

df_mm=pd.DataFrame(df_mm,columns=["days_since_review_count_day"])


hotels["days_since_review_count_day_MinMaxScaler"]=df_mm["days_since_review_count_day"]

# Разберём hotel_name — название отеля



In [ ]:
hotel_name_hotels_value_counts=hotels["hotel_name"].value_counts().sort_values(ascending=False)

# Отелей слишком много. Нужно отсеять часть названий которые встречаются слишком редко .
# Оставим только не отели которые встречаются почаще. А именно возьмём отели которые стречаются в 5% строк



In [ ]:
DF_For_Coding=(hotel_name_hotels_value_counts[hotel_name_hotels_value_counts>hotels.shape[0]*0.05])



нет на столько популярного отеля который бы фигурировал в  5% строк
Поэтому просто возьмём первые 10 отелей  - типо топ . И создадим флаг популярности.

In [ ]:
DF_For_Coding=(hotel_name_hotels_value_counts[hotel_name_hotels_value_counts>1000])
DF_For_Coding.sum()

list_famous_Hotels=DF_For_Coding.index
list_famous_Hotels

hotels["hotel_name_famous_bool"]=hotels["hotel_name"].isin( list_famous_Hotels)


hotels["hotel_name_famous"]=0
hotels.loc[hotels[hotels["hotel_name_famous_bool"]].index,["hotel_name_famous"]]=1
hotels.loc[hotels[hotels["hotel_name_famous_bool"]==False].index,["hotel_name_famous"]]=0
hotels["hotel_name_famous"]=hotels["hotel_name_famous"].astype(int)



# Разберём hotel_address  -Нужно достать название страны и Города

In [ ]:
# У нас есть столбец со страной проживания рецензента возьмём из него все уникальные значения

list_country=hotels["reviewer_nationality"].value_counts().index
# Уберём пробелы в начале и конце

def strip_str (string :str):
  if string!='':
       return string.strip()
list_country=list(map (strip_str,list_country))


Посмотрим на топ стран из которых родом рецензенты

In [ ]:
df_for_figure=hotels["reviewer_nationality"].value_counts().sort_values(ascending=False)[:5]
fig=plt.figure(figsize=(8,4))
axes=fig.add_axes([0,0,1,1])
axes.bar(x=df_for_figure.index ,height=df_for_figure)


Видно что у нас  половина рецензентов из Англии добавим флаг англичанина

In [ ]:
hotels["reviewer_nationality_United_Kingdom"]=hotels["reviewer_nationality"].apply(lambda x : 1 if x.lower().replace(" ","")=="United Kingdom".lower().replace(" ","")  else 0)



In [ ]:
# Список стран Рецензентов код представленный ниже долго выполняется

# чтоб сократить время на частые запуски выполним код котороый ниже 1 раз и надём все страны отели котрых предсталены в таблице



list_country=["United Kingdom","Spain","France","Netherlands","Austria","Italy"]

# Теперь будем перебирать небольшой список стран

In [ ]:
hotels["hotel_address"]

# теперь будем проверять наличие названия страны в адресе
# и если название есть то укажем название страны

# функция для проверки названия страны


def take_country(hotel_address, country_name):
      if country_name.lower().replace(" ","") in hotel_address.lower().replace(" ",""):
        return  country_name

hotels["hotel_address_country"]=None


for country_name in list_country:

    hotels.loc[hotels[hotels["hotel_address_country"].isnull()].index,["hotel_address_country"]]=hotels["hotel_address"].apply(take_country,country_name=country_name)





In [ ]:
hotels["hotel_address_country"].value_counts()

In [ ]:
# У нас всего 5 стран для проверки
# это значит 1 мы можем достать название города так как  принцеп формирования адреса в разных странах свой папример у United Kingdom  после города есть 2 3х символьные конструкции
# У Spain перед городом есть 5 цифр
# У France перед городом есть 5 цифр
# У Netherlands перед городом есть 4  цифры
# У Austria перед городом есть 4  цифры
# У Italy перед городом есть 5  цифр

# hotels[hotels["hotel_address_country"]=="Italy"]["hotel_address"].iloc[4545]

In [ ]:
# Данная функция  будет вытаскивать  названрие города + какойто признак района

def take_city(row):
      Country_name=row["hotel_address_country"]
      hotel_address=row["hotel_address"]
      City_name="Ошибка поиска города"

      if Country_name=="United Kingdom":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-4:-2]
        City_name=" ".join(City_name)


      if Country_name=="Spain":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-2:]
        City_name=" ".join(City_name)



      if Country_name=="France":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-2:]
        City_name=" ".join(City_name)


      if Country_name=="Netherlands":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-3:]
        City_name=" ".join(City_name)

      if Country_name=="Austria":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-2:]
        City_name=" ".join(City_name)


      if Country_name=="Italy":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-2:]
        City_name=" ".join(City_name)


      return City_name



# hotel_address ="21 Piccadilly Westminster Borough London W1J 0BH United Kingdom"
# Country_name="United Kingdom"


# hotel_address ="Calabria 90 92 Eixample 08015 Barcelona Spain"
# Country_name="Spain"


# hotel_address ="20 Rue Jean Rey 15th arr 75015 Paris France"
# Country_name="France"

# hotel_address ="IJdock 6 Amsterdam City Center 1013 MM Amsterdam Netherlands"
# Country_name="Netherlands"


# hotel_address ="Schlossallee 8 14 Penzing 1140 Vienna Austria"
# Country_name="Austria"

# hotel_address ="Piazza Duca D Aosta 4 6 Central Station 20124 Milan Italy"
# Country_name="Italy"




# take_city(hotel_address , Country_name)

In [ ]:
hotels["hotel_address_city_name"]=hotels.apply( take_city,axis=1 )




In [ ]:
hotels[["hotel_address_city_name"]].value_counts()

In [ ]:
# Закодируем название страны Бинарным кодированием
bin_encoder=ce.OneHotEncoder(cols=["hotel_address_country"],use_cat_names=True)

type_bin=bin_encoder.fit_transform(hotels["hotel_address_country"])
hotels=pd.concat([hotels,type_bin],axis=1)

# lat — географическая широта отеля;
# lng — географическая долгота отеля.


## Нужно заполнить пропуски

In [ ]:
hotels["lng"].isnull().sum()

In [ ]:
hotels[["hotel_address","hotel_address_city_name","lat","lng"]][hotels["lng"].isnull()]["hotel_address_city_name"].value_counts()

In [ ]:
# Из ранее полученного признака Города мы можем достать адрес  и заполнить пропущенную геолокацию

# 1 Получем все города   с пропусками в гео локации

lat_nulls=hotels[hotels["lat"].isnull()]["hotel_address_city_name"].unique()

# 2 Получим для городов указынных выше их гео локации

df_merge_lat_not_null=hotels[(hotels["lat"].isnull()==False) & (hotels["hotel_address_city_name"].isin (lat_nulls) )][["hotel_address_city_name","lat","lng"]]


# 3 получим моду для каждого  города
df_merge_lat_not_null=df_merge_lat_not_null.groupby(by=["hotel_address_city_name"]).agg(lambda x :x.mode())



In [ ]:

# 4 В таблице отелей заполним пропуски в гео локации
# так как  значений не много  заполним пропуски используя цикл for

for row_Megge_table in df_merge_lat_not_null.index:

    hotels.loc[  hotels[(hotels["lat"].isnull())& (hotels["hotel_address_city_name"]==row_Megge_table)].index ,  "lat"     ]=(df_merge_lat_not_null.loc[row_Megge_table,"lat"])
    hotels.loc[  hotels[(hotels["lng"].isnull())& (hotels["hotel_address_city_name"]==row_Megge_table)].index ,  "lng"     ]=(df_merge_lat_not_null.loc[row_Megge_table,"lng"])

# Пропуски в столбцах lat  lng  -заполнены

In [ ]:
# Уберём у Признака города - индекс района
def take_city_DELL_INDEX(row):
      Country_name=row["hotel_address_country"]
      hotel_address=row["hotel_address"]
      City_name="Ошибка поиска города"

      if Country_name=="United Kingdom":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-3:-2]
        City_name=" ".join(City_name)


      if Country_name=="Spain":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-1:]
        City_name=" ".join(City_name)



      if Country_name=="France":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-1:]
        City_name=" ".join(City_name)


      if Country_name=="Netherlands":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-1:]
        City_name=" ".join(City_name)

      if Country_name=="Austria":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-1:]
        City_name=" ".join(City_name)


      if Country_name=="Italy":
        City_name=hotel_address.replace(Country_name,"").strip().split(" ")[-1:]
        City_name=" ".join(City_name)


      return City_name



In [ ]:
hotels["hotel_address_city_name"]=hotels.apply( take_city_DELL_INDEX,axis=1 )


In [ ]:
# Закодируем название города Бинарным кодированием
bin_encoder=ce.BinaryEncoder(cols=["hotel_address_city_name"])

type_bin=bin_encoder.fit_transform(hotels["hotel_address_city_name"])
hotels=pd.concat([hotels,type_bin],axis=1)

# Разберём positive_review — положительный отзыв, который рецензент дал отелю

Среди отзывов есть  упоминание

Location - локации

Everything  - всё  

Breakfast  - завтрак

это кондидаты стать отдельными признаками

но для начала надо поработать с текстом и достать их

Так как в одной строке может быть несколько


In [ ]:

# Подправим количество слов в столбца где присутстсвутет no positive  - вначале найдём все синонимы а потом изменим на no positive  а дальше заменим кол-во слов на 0

hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().strip())
list_no_positive=["nothing","n a","none"]
hotels.loc[hotels[hotels["positive_review"].isin(list_no_positive)].index,["positive_review"]]="no positive"
hotels.loc[hotels[hotels["positive_review"]=="no positive"].index,["review_total_positive_word_counts"]]=0

hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("the ","").replace("and ","").replace("hotel","").strip())




# Избавимся от слов качества  comfy friendly  comfortable   helpful

# Заменим редко встречающиеся прилогательные на часто
# comfy  и  comfortable и helpful  на good

hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("comfortable ","good").replace("helpful ","good").replace("friendly ","good").replace("comfy ","good").replace( "nice ","good").replace("very good","great") .replace( "fantastic ","great")  .replace( "amazing ","great").strip())

# Добавим столбцы для эмоцианальной окраски  -т.е
# соберём словарь с эмоциональной оценкой


dictHappy={"good":1,
           "great":2,
           "perfect":3,
            "excellent":4
}

hotels["positive_review_level_of_joy"]=1
hotels["positive_review_level_of_joy"]=hotels["positive_review"].apply(lambda x: 2 if "great" in x else  3 if "perfect" in x else  4 if  "excellent" in x else 1)
# Для строк где нет положительного комментария проставим 0
hotels.loc[hotels[hotels["positive_review"]=="no positive"].index,["positive_review_level_of_joy"]]=0
# теперь избавимся от слов эмоциональной окраски в комментариях

# Регулярным вывражением избавимся от определений is great  например
Regex_pattren="((was)|(is)|(were)|(are)) [a-z]{1,}"
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: re.sub(Regex_pattren, "",  x, count=0))



hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("good","").replace("great","").replace("perfect","").replace("excellent","").replace("  "," ").strip())

# У некоторых комментарием есть синонимы - подведём их под единое определение

hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("every thing","everything").replace("all","everything").replace("position","location").replace("rooms","room").replace("beds","bed").replace("food","breakfast").replace("comfort","room").replace("cleanliness","clean").replace("service","staff").strip())





# Вынесем самые популярные теги комментарии в отдельные столбцы

hotels["positive_review_location"]=hotels["positive_review"].apply(lambda x: 1 if "location" in x else 0 )
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("location","").strip())



hotels["positive_review_everything"]=hotels["positive_review"].apply(lambda x: 1 if "everything" in x else 0 )
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("everything","").strip())


hotels["positive_review_Breakfast"]=hotels["positive_review"].apply(lambda x: 1 if "breakfast" in x else 0 )
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("breakfast","").strip())


hotels["positive_review_staff"]=hotels["positive_review"].apply(lambda x: 1 if "staff" in x else 0 )
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("staff","").strip())

hotels["positive_review_staff"]=hotels["positive_review"].apply(lambda x: 1 if "clean" in x else 0 )
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("clean","").strip())

hotels["positive_review_staff"]=hotels["positive_review"].apply(lambda x: 1 if "room" in x else 0 )
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("room","").strip())

hotels["positive_review_staff"]=hotels["positive_review"].apply(lambda x: 1 if "bed" in x else 0 )
hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("bed","").strip())

# Избавимся от упоминание ранее добавленных слов
# hotels["positive_review"]=hotels["positive_review"].apply(lambda x: x.lower().replace("location","").replace("everything","").replace("breakfast","").replace("staff","").strip())


hotels["positive_review"].value_counts().head(30)

# Разберём negative_review — отрицательный отзыв, который рецензент дал отелю;

In [ ]:

hotels["negative_review"].value_counts().head(30)
hotels.groupby(by=["negative_review"])["review_total_negative_word_counts"].agg(['median','count']).sort_values(by=['count'],ascending=False).head(30)

In [ ]:
# Среди негативных комментариев есть положительные комментарии с эмоциональной окраской

hotels["negative_review_level_of_joy"]=hotels["negative_review"].apply(lambda x: 2 if "great" in x else  3 if "perfect" in x else  4 if  "excellent" in x else 0)




In [ ]:
# Перенесём все положительные оценки в ранее созданный столбец positive_review_level_of_joy
hotels["positive_review_level_of_joy"]=hotels.apply(lambda Row: Row["negative_review_level_of_joy"] if Row["negative_review_level_of_joy"]>Row["negative_review_level_of_joy"] else Row["positive_review_level_of_joy"]  ,axis =1)

In [ ]:



# Заменим комментарии которые положительные  являются нейтральными  но почемуто попали в столбец негативных

hotels["negative_review"]=hotels["negative_review"].apply(lambda x: x.lower().strip())
list_no_negative=["nothing","n a","none","nothing really","all good","nothing at all","na","liked everything"
,"i liked everything","everything was good","everything was perfect","nothing to dislike","everything was great","non","nothing it was perfect","everything was fine"
,"all was good","nothing all good","nothing everything was perfect","absolutely nothing","no complaints","nothing everything was great"]

hotels.loc[hotels[hotels["negative_review"].isin(list_no_negative)].index,["negative_review"]]="no negative"
hotels.loc[hotels[hotels["negative_review"]=="no negative"].index,["review_total_negative_word_counts"]]=0

In [ ]:
# Для удобства поиска напишем функцию которая будет находить все строки в которых существует отпределённое слово
def find_text(df,coll_name,find_str):
    set_str=set(df[coll_name].apply(lambda x : x if find_str in x else "" ))
    return set_str




In [ ]:

# Заменим the  на пустоту
hotels["negative_review"]=hotels["negative_review"].apply(lambda x: x.lower().replace("the ","").strip())

# Заменим синонимы слова everything  на него же
hotels["negative_review"]=hotels["negative_review"].apply(lambda x: x.lower().replace("nothing not to like","everything").strip())

hotels["negative_review_location"]=hotels["negative_review"].apply(lambda x: 1 if "location" in x else 0 )
# hotels["negative_review"]=hotels["negative_review"].apply(lambda x: x.lower().replace("location","").strip())


# hotels["negative_review"]=hotels["negative_review"].apply(lambda x: x.lower().replace("small rooms","room").replace("small room","room").replace("room size","room").replace("room was very small","room").replace("room very small","room").replace("room was small","room").replace("room size","room").strip())
hotels["negative_review_room"]=hotels["negative_review"].apply(lambda x: 1 if "room" in x else 0 )
# hotels["negative_review"]=hotels["negative_review"].apply(lambda x: x.lower().replace("room","").strip())



In [ ]:
hotels["negative_review"].value_counts().head(30)
hotels.groupby(by=["negative_review"])["review_total_negative_word_counts"].agg(['mean','count']).sort_values(by=['count'],ascending=False).head(30)


# Разберём признаки количество слов в отзыве
* review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
* review_total_positive_word_counts — общее количество слов в положительном отзыве.

In [ ]:
# С признаком количества слов желательно что то сделать  т.к  в нем среднее значение слов в отзыве 25 а  максимальное в несколько раз больше Проведём нормализацию RobustScaler

r_scaler=preprocessing.RobustScaler()

df_r=r_scaler.fit_transform(hotels[["review_total_positive_word_counts"]])

df_r = pd.DataFrame(df_r, columns=["review_total_positive_word_counts_RobustScaler"])

# Но так как нам нужны только положительные отзывы сместим результат в право чтоб самое минимальное значение было нулём

hotels['review_total_positive_word_counts_RobustScaler']=(df_r["review_total_positive_word_counts_RobustScaler"]+np.abs(df_r["review_total_positive_word_counts_RobustScaler"].min()))


In [ ]:
# С признаком количества слов желательно что то сделать  т.к  в нем среднее значение слов в отзыве 25 а  максимальное в несколько раз больше Проведём нормализацию RobustScaler

r_scaler=preprocessing.RobustScaler()

df_r=r_scaler.fit_transform(hotels[["review_total_negative_word_counts"]])

df_r = pd.DataFrame(df_r, columns=["review_total_negative_word_counts_RobustScaler"])

# Но так как нам нужны только положительные отзывы сместим результат в лево  чтоб самое большое значение было 0
hotels['review_total_negative_word_counts_RobustScaler']=(df_r["review_total_negative_word_counts_RobustScaler"]-np.abs(df_r["review_total_negative_word_counts_RobustScaler"].max()))



# Разберём additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.

In [ ]:
# Добавим коэффициент показывающий сооношение общего количества отзывов и количества отзывов без комментария

# 1 Поймём за какой период были отзывы без коммента

hotels.groupby(by=["hotel_address"])[["additional_number_of_scoring","total_number_of_reviews_reviewer_has_given","total_number_of_reviews"]].agg([ 'min',"max",'count']).sort_values(by=[("total_number_of_reviews_reviewer_has_given","count")],ascending=False)

In [ ]:
# Соберём табилицу название отеля и коэф  который равен   additional_number_of_scoring   делённый на сумму (additional_number_of_scoring + total_number_of_reviews_reviewer_has_given + количество комментариев для данного отеля )

MergeTable=hotels.groupby(by=["hotel_name"])[["additional_number_of_scoring","total_number_of_reviews_reviewer_has_given","total_number_of_reviews","average_score"]].agg([ 'min','max','count',"nunique"])
MergeTable["additional_number_of_scoring_total_number_of_reviews"]=    MergeTable[("additional_number_of_scoring","min")] /(      + MergeTable[("total_number_of_reviews","min")])
MergeTable.sort_values(by=["additional_number_of_scoring_total_number_of_reviews"],ascending=False)

MergeTable=MergeTable["additional_number_of_scoring_total_number_of_reviews"].sort_values()

In [ ]:
# Теперь смержим основную таблицу и коффициент соотношения additional_number_of_scoring   к  total_number_of_reviews
hotels=hotels.merge(MergeTable,left_on=["hotel_name"],right_index=True)

In [ ]:
hotels[["hotel_name","additional_number_of_scoring","total_number_of_reviews","additional_number_of_scoring_total_number_of_reviews"]].sort_values(by=["additional_number_of_scoring","additional_number_of_scoring_total_number_of_reviews"])

# Разберём hotel_name — название отеля;

In [ ]:
hotels["hotel_name"].value_counts()>hotels.shape[0]*0.01
# hotel_name просто удалим  т.к тут нет  часто встречающихся отелей

# Разберём reviewer_nationality — страна рецензента



In [ ]:
# Поработаем с признаком reviewer_nationality — страна рецензента;

hotels["reviewer_nationality"].value_counts()
# Закодируем признак страны бинарным кодированием


bin_encoder=ce.BinaryEncoder(cols="reviewer_nationality")
bin_encoder
type_bin=bin_encoder.fit_transform(hotels["reviewer_nationality"])
hotels=pd.concat([hotels,type_bin],axis=1)



In [ ]:
# добавим признак землячества - т.е если в адресе отеля есть название  страны речензента поставим 0  если нет то 1
hotels["hotel_address_home"]=  hotels.apply (lambda row: 0 if  row["reviewer_nationality"].lower() in row["hotel_address"].lower() else 1 , axis=1)

In [ ]:
hotels[["hotel_address_home","reviewer_nationality","hotel_address"]]

# Удалим лишние столбцы которые больше не понадобятся

In [ ]:


# Удалим вспомогательный столбецы
hotels=hotels.drop(["index_from_hotels","hotel_name_famous_bool"],axis=1)


# УДалим столбец Тегов
hotels.drop(["tags","tags_list"],inplace=True,axis=1)


# Удалим признаки  Появившиеся при работе с датой review_date и  days_since_review


hotels.drop(["review_date","review_date_year","review_date_day_of_week","review_date_month"],inplace=True,axis=1)

# Удалим название отеля

hotels.drop(["hotel_name"],inplace=True,axis=1)

# Удалим review_total_negative_word_counts  и  review_total_positive_word_counts

hotels.drop(["review_total_positive_word_counts","review_total_negative_word_counts"],inplace=True,axis=1)


# Удалим   days_since_review  и  days_since_review_count_day
hotels.drop(["days_since_review","days_since_review_count_day"],inplace=True,axis=1)

# Удалим   additional_number_of_scoring  и  total_number_of_reviews
hotels.drop(["additional_number_of_scoring","total_number_of_reviews"],inplace=True,axis=1)


In [ ]:
# Удалим столбцы с типом данных object

drop_list=hotels.select_dtypes(['object']).columns

hotels.drop(drop_list,axis=1,inplace=True)



hotels.head()


# Отбор признаков .
Если какието признаки слишком сильно корелируют друг с другом то один из них нужно удалять

После того как было создано много признаков и закодированно их надо разделить на  категориальные и числовые

In [ ]:
# Признаков много поэтому мы не можем использовать тепловую карту она не информативна
# Удалим из таблици целевой признак

hotels_corr=hotels.drop(['reviewer_score'],axis=1).corr(method='spearman')[hotels.corr()!=1]



hotels_corr_max=hotels_corr.max().abs().sort_values(ascending=False)



In [ ]:

hotels_corr_max=hotels_corr_max[hotels_corr_max>0.7]


# В таблицу с максимальными кореляциями могут попасть столбцы которые появились за счёт бинарного кодирования  - это не хорошо  такие столбцы надо из нее исключить
# Найти такой признак можно  с помощью Regex  их шаблон следующий у них на конце есть  _число
Regex_pattern=".{1,}_\d{1,}$"

index_hotels_corr_max=hotels_corr_max.index
index_hotels_corr_max=pd.DataFrame(index_hotels_corr_max)
index_hotels_corr_max["bin_collstatus"]=index_hotels_corr_max[0].apply(lambda x  :   re.fullmatch(Regex_pattern,   x) is None)
index_hotels_corr_max=index_hotels_corr_max[index_hotels_corr_max["bin_collstatus"]][0]

# теперь  в выборку не будут попадать столбцы задействованные в бинарном кодировании

In [ ]:
index_hotels_corr_max

У нас есть  4 скорелированных признака  ,

но  Tags_Count  имеет сильную кореляцию  с Tags_Submitted from a mobile device    но  Tags_Count  -числовой а Tags_Submitted from a mobile device  -Бинарный

hotel_address_country_United Kingdom  корелирует  с признаком additional_number_of_scoring_total_number_of_reviews    тут так же  hotel_address_country_United Kingdom  - бинарный а  additional_number_of_scoring_total_number_of_reviews  -непрерывный - числовой

lat -сильно корелирует с признаком hotel_address_city_name_2  - это признак часть бинарного кодирования  

In [ ]:
hotels_corr['lat'].sort_values()

In [ ]:
ColForDel=hotels_corr_max[hotels_corr_max>0.7].index
ColForDel

так как ранее  мы основывались на значении максимальной кореляции  то первой строкой всегда будет признак с которым данный столбец корелирует сильнее всего

но один признак может сильно корелировать и с другими признаками - поэтому нужно  проверить 2 условия
1  Со сколькими признаками сильно корелирует текущий признак (>0.7)
2 Если количество признаком с сильной кореляцией одинаково в сравнении с аналогичным значением у признака который с текущим корелирует максимально
то  мы найдём сумму кореляций с другими  признаками и в "живых" останется тот у кого она больше

In [ ]:
# надо выбрать

hotels_corr["Tags_Count"].abs().sort_values(ascending=False)




# Раздел с моделью

In [ ]:
# Вывести столбцы с пропусками

hotels.isnull().sum().sort_values().sum()

In [ ]:
# Соберём в словарь значения средних арефметических для числовых признаков

dict_for_fillna=dict(hotels[hotels.select_dtypes(['int64', 'float64']).columns].mean())


In [ ]:
# Заполним пропусвки  средним арефметическим
hotels.fillna(dict_for_fillna,inplace=True)

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)
X = hotels.drop(['reviewer_score'], axis = 1)
y = hotels['reviewer_score']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
y=y.astype('int')

# Оценим влияние категориальных признаков на целевую переменную

In [ ]:
# ОЦеним влияние Tags

List_FamosTags=[

"Tags_Leisure trip" ,
"Tags_Submitted from a mobile device",
"Tags_Couple",
"Tags_Stayed 1 night",
"Tags_Stayed 2 nights",
"Tags_Solo traveler",
"Tags_Stayed 3 nights",
"Tags_Business trip",
"Tags_Group",
"Tags_Family with young children",
"Tags_Stayed 4 nights",
"Tags_Double Room",
"Tags_Standard Double Room",
"Tags_Superior Double Room",
"Tags_Family with older children"
]

cat_cols =List_FamosTags


from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
List_review_date= [
 "review_date_day_of_week_Friday",
 "review_date_day_of_week_Thursday",
 "review_date_day_of_week_Tuesday",
 "review_date_day_of_week_Saturday",
 "review_date_day_of_week_Wednesday",
 "review_date_day_of_week_Monday",
 "review_date_day_of_week_Sunday",
 "review_date_month_February",
 "review_date_month_January",
 "review_date_month_October",
 "review_date_month_September",
 "review_date_month_March",
 "review_date_month_December",
 "review_date_month_May",
 "review_date_month_November",
 "review_date_month_June",
 "review_date_month_July",
 "review_date_month_April",
 "review_date_month_August"
 ]

cat_cols =List_review_date




imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
List_hotel_name= ["hotel_name_famous",
"hotel_address_country_United Kingdom",
"hotel_address_country_France",
"hotel_address_country_Netherlands",
"hotel_address_country_Italy",
"hotel_address_country_Austria",
"hotel_address_country_Spain",
"hotel_address_city_name_0",
"hotel_address_city_name_1",
"hotel_address_city_name_2"]



cat_cols =List_hotel_name




imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
List_positive_review= ["positive_review_level_of_joy",
"positive_review_location",
"positive_review_everything",
"positive_review_Breakfast",
"positive_review_staff",
"negative_review_level_of_joy",
"negative_review_location",
"negative_review_room"]

cat_cols =List_positive_review




imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')



In [ ]:

List_positive_reviewer_nationality= ["reviewer_nationality_0",
"reviewer_nationality_1",
"reviewer_nationality_2",
"reviewer_nationality_3",
"reviewer_nationality_4",
"reviewer_nationality_5",
"reviewer_nationality_6",
"reviewer_nationality_7"]

cat_cols =List_positive_reviewer_nationality




imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')


In [ ]:
# Достанем из таблицы Отелей  все столбцы которые являются категориальными - это не сложно  -Количество уникальных значений в  них 2   это 0 и 1

List_CAtColls=hotels.nunique()[hotels.nunique()<=2].index


In [ ]:
# List_CAtColls=[

# "Tags_Leisure trip" ,
# "Tags_Submitted from a mobile device",
# "Tags_Couple",
# "Tags_Stayed 1 night",
# "Tags_Stayed 2 nights",
# "Tags_Solo traveler",
# "Tags_Stayed 3 nights",
# "Tags_Business trip",
# "Tags_Group",
# "Tags_Family with young children",
# "Tags_Stayed 4 nights",
# "Tags_Double Room",
# "Tags_Standard Double Room",
# "Tags_Superior Double Room",
# "Tags_Family with older children",
#  "review_date_day_of_week_Friday",
#  "review_date_day_of_week_Thursday",
#  "review_date_day_of_week_Tuesday",
#  "review_date_day_of_week_Saturday",
#  "review_date_day_of_week_Wednesday",
#  "review_date_day_of_week_Monday",
#  "review_date_day_of_week_Sunday",
#  "review_date_month_February",
#  "review_date_month_January",
#  "review_date_month_October",
#  "review_date_month_September",
#  "review_date_month_March",
#  "review_date_month_December",
#  "review_date_month_May",
#  "review_date_month_November",
#  "review_date_month_June",
#  "review_date_month_July",
#  "review_date_month_April",
#  "review_date_month_August",
#  "hotel_name_famous",
# "hotel_address_country_United Kingdom",
# "hotel_address_country_France",
# "hotel_address_country_Netherlands",
# "hotel_address_country_Italy",
# "hotel_address_country_Austria",
# "hotel_address_country_Spain",
# "hotel_address_city_name_0",
# "hotel_address_city_name_1",
# "hotel_address_city_name_2",
# "positive_review_level_of_joy",
# "positive_review_location",
# "positive_review_everything",
# "positive_review_Breakfast",
# "positive_review_staff",
# "negative_review_level_of_joy",
# "negative_review_location",
# "negative_review_room",
# "reviewer_nationality_0",
# "reviewer_nationality_1",
# "reviewer_nationality_2",
# "reviewer_nationality_3",
# "reviewer_nationality_4",
# "reviewer_nationality_5",
# "reviewer_nationality_6",
# "reviewer_nationality_7"]


print(len(List_CAtColls))

cat_cols =List_CAtColls

# chi2(X[cat_cols], y)[0]


from sklearn.feature_selection import chi2 # хи-квадрат

# Из всех признаков возьмём только 15 первых самых влиятельных

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols).sort_values(ascending=False).iloc[0:16]
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
imp_cat

Возьмём первые 15  признаков по влиянию на целевую переменную
Среди этих признаков есть признаки которые учавствовали в бинарном кодировании. Надо забрать признак со всеми его столбцами

In [ ]:
# Достанем из столбца самых влиятельных  признаков признаки из Бинарного кодироввания
# Они имеют шаблон  у них на конце всегда есть число

Regex_pattern=".{1,}_\d{1,}$"



imp_cat=pd.DataFrame(imp_cat)

# index_hotels_corr_max=hotels_corr_max.index
# index_hotels_corr_max=pd.DataFrame(index_hotels_corr_max)
# index_hotels_corr_max["bin_collstatus"]=index_hotels_corr_max[0].apply(lambda x  :   re.fullmatch(Regex_pattern,   x) is None)
# index_hotels_corr_max=index_hotels_corr_max[index_hotels_corr_max["bin_collstatus"]][0]
imp_cat["index_names"]=imp_cat.index

imp_cat["din_colls"]=imp_cat["index_names"].apply(lambda x  :   re.fullmatch(Regex_pattern,   x) is None)

# Получим список тех столбцов в которые учавствовали в бинарном кодировании
list_of_bin_index=imp_cat[imp_cat["din_colls"]==False].index


# Теперь "оторвём" у них число
Regex_pattern_2="_\d{1,}$"


imp_cat.loc[list_of_bin_index,"index_names"]=imp_cat.loc[list_of_bin_index,"index_names"].apply(lambda x: re.sub(Regex_pattern_2, "", x , count=0)  )

# сОБЕРЁМ ВО МНОЖЕСТВО ПОЛУЧИВШИЕСЯ НАЗВАНИЯ

set_of_bin_index=set(imp_cat[imp_cat["din_colls"]==False]["index_names"])




In [ ]:
# Теперь достанем все названия столбцов  из бинарного кодирования соответствущие данному шаблону

itog_list=[]

name_Bin_Coll='reviewer_nationality'


for name_Bin_Coll  in set_of_bin_index:

  Regex_pattern=name_Bin_Coll+"_\d{1,}$"
  pattern=re.compile(Regex_pattern)

  cleaned_list=[item for item in hotels.columns if  pattern.match(item)]
  itog_list=itog_list + cleaned_list



In [ ]:
# Соберём в список  все названия столбцов которые  максимально влияют на целевой столбец

List_itog_bin_CaT_coll=list(set(itog_list)|set(imp_cat.index))

List_itog_bin_CaT_coll

# Оценим влияние непрерывных признаков на целевую переменную

In [ ]:
# Достанем из таблицы оставшиеся  столбцы и убедимся что остались только не прерывные признаки

list_Num_Colls=set(hotels.columns)-set(List_CAtColls)
list_Num_Colls.remove('reviewer_score')
list_Num_Colls=list(list_Num_Colls)

In [ ]:
from sklearn.feature_selection import f_classif # anova

num_cols = list_Num_Colls


imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Так как на надо выбрать только влиятельные признаки то будем сравнивать их со среднем аремфетическим - так отстоющие точно отсеятся

In [ ]:
# Заберём только те столбцы которые больше стредненего аремфметичкеского умноженного на 0.5
list_Num_Colls=list(imp_num[imp_num>imp_num.mean()*0.5].index)
# И добавим тестовый флаг


In [ ]:
# Соберём в один список все элементы

itog_list=List_itog_bin_CaT_coll+list_Num_Colls


In [ ]:

if ("TestFlag" in  itog_list)==False:
    print("Добавим тестовый флаг если его не было в итоговом списке столбцов ")
    list_Num_Colls.append("TestFlag")

In [ ]:
# теперь округлим числовые признаки до 2 знаков после запятой
hotels[list_Num_Colls]=hotels[list_Num_Colls].round(2)

# Заного переопределим модель

In [ ]:
itog_list.append("reviewer_score")

# Переопределяем таблицу отели

hotels=hotels[itog_list]



In [ ]:


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)


# Разделим признаки на тестовые и тренировачные
train_hotels=hotels[hotels["TestFlag"]==0].drop(['TestFlag'], axis=1)



test_hotels=hotels[hotels["TestFlag"]==1].drop(['TestFlag'], axis=1)
# Оставим в X только ранее отобранные признаки
X = train_hotels.drop(['reviewer_score'], axis=1)
y = train_hotels['reviewer_score']


# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)


MAPE: 0.14138286324223787  -было - чем меньше тем лучше
0.10-  это хороший результат

0.1356038539096172  - переработал теги

0.1348252196321253  - добавил признак накрутки оценки


разобрали признак города  0.13484628373241422

разбор признака  положительный  комментарий    0.13198419052564622

разбор признака  отрицательный   комментарий 0.12936184744934814


Приравнивание выбросов к 1 й и 3й квантили сответственно 0.13551607852346315

После удаления признаков и приравнения оставшихся выбросов к 1й и 3й квантили 0.12888070628273923

Просто удаление  лишних  строк 0.12823455796930133



In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
test_hotels

In [ ]:
test_hotels=test_hotels.drop(['reviewer_score'], axis=1)
predict_submission = regr.predict(test_hotels)


In [ ]:
sample_submission = pd.read_csv('/kaggle/input/sf-booking/submission.csv')

# Запись в целевой признак наших расчетов
sample_submission['reviewer_score'] = predict_submission

# Cохраняем результат в датасет
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)